In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
torch.set_float32_matmul_precision('medium')

In [ ]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40

path = "../data/ModelNet40"
categories = ['bottle', 'car']

tr, te = get_dataloaders(path, categories=categories)

Loading renders for car:  86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                    | 169/197 [00:25<00:04,  6.77it/s]

In [ ]:
next(iter(te))["render-features"].shape

In [ ]:
import models

m = models.SPVD()

lr = 1e-4
model = models.DiffusionBase(m, lr=lr)

In [ ]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("../logs/ModelNet40/bottle-car", name="test")

checkpoint_name = "-".join(categories) if categories else "all"
epochs = 100
checkpoint_callback = ModelCheckpoint(
    dirpath=f'../checkpoints/ModelNet40/{checkpoint_name}',
    filename="{epoch:02d}-{val_loss:.2f}", 
    every_n_epochs=5
)

trainer = L.Trainer(
    logger=logger,
    log_every_n_steps=5,
    max_epochs=epochs, 
    gradient_clip_val=10.0, 
    callbacks=[checkpoint_callback]
)

In [ ]:
t = te.dataset
t.render_features[0][0].shape

In [ ]:
trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)

In [11]:
from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

preds = ddpm_sched.sample(model.cuda(), 16, 2048)

from utils.visualization import visualize_notebook
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
